In [ ]:
Вспомним схему БД для библиотеки:
Reader( ID, LastName, FirstName, Address, BirthDate)
Book ( ISBN, Title, Author, PagesNum, PubYear, PubName)
Publisher ( PubName, PubKind)
Category ( CategoryName, ParentCat)
Copy ( ISBN, CopyNumber, Shelf, Position)
Borrowing ( ReaderNr, ISBN, CopyNumber, ReturnDate)
BookCat ( ISBN, CategoryName )  


Напишите SQL-запросы, возвращающие следующие данные:

Все фамилии (LastName) читателей (Reader) из Москвы
Все названия (Title) и авторов (Author) книг (Books), опубликованных издателями (Publisher) научной или справочной литературы (pubKind либо 'Science', либо 'Reference')
Все названия (Title) и авторов (Author) книг (Books), которые брал Иван Иванов.
Все идентификаторы (ISBN) книг (Book), относящихся к категории "Mountains", но не относящихся к категории "Travel". Подкатегории не учитывать.
Все фамилии и имена читателей, которые вернули хотя бы одну книгу (Borrowing.ReturnDate is not null)
Все фамилии и имена читателей, которые брали (Borrowing)  хотя бы одну книгу (Book), которую брал Иван Иванов. Ответ не должен содержать самого Ивана Иванова.

In [ ]:
select lastname from reader r where address like '%Москва%'

select title, author, ubname
from book b 
join publisher p on b.PubName=p.PubName 
where pubKind in ('Science', 'Reference')

select title, author
from book b
join borrowing br on b.isbn=br.isbn
join reader r on br.readernr=r.id
where r.firstname = 'Иван' and r.lastname = 'Иванов'

select b.isbn
from book b
join bookcat bc on b.isbn=bc.isbn
where bc.categoryname = 'Mountains' 
except 
select b.isbn 
from book b
join bookcat bc on b.isbn=bc.isbn 
where bc.categoryname = 'Travel'



select distinct r.lastname, r.firstname
from book b
join borrowing br on b.isbn=br.isbn
join reader r on br.readernr=r.id
where br.returndate is not null


select r.lastname, r.firstname
from book b
join borrowing br on b.isbn=br.isbn
join reader r on br.readernr=r.id
where br.isbn in (
	select (br.isbn)
	from book b
	join borrowing br on b.isbn=br.isbn
	join reader r on br.readernr=r.id
	where r.lastname='Иванов' and r.firstname='Иван'
	)
except 
select (r.lastname, r.firstname)
from book b
join borrowing br on b.isbn=br.isbn
join reader r on br.readernr=r.id
where r.lastname='Иванов' and r.firstname='Иван'


select * from borrowing b 

select * from book

select * from reader r 




In [ ]:
Задание 1. 
Вспомним схему для библиотеки

Reader( ID, LastName, FirstName, Address, BirthDate)
Book ( ISBN, Title, Author, PagesNum, PubYear, PubName)
Publisher ( PubName, PubKind)
Category ( CategoryName, ParentCat)
Copy ( ISBN, CopyNumber, Shelf, Position)
Borrowing ( ReaderNr, ISBN, CopyNumber, ReturnDate)
BookCat ( ISBN, CategoryName )  


Напишите запросы:

Напишите SQL-запросы, возвращающие следующие данные:

Вывести все названия (Title) книг (Book) вместе с названиями (PubName) их издателей (Publisher)
Вывести ISBN книги/всех книг (Book) с максимальным количеством страниц (PagesNum)
Какие авторы (Author) написали больше пяти книг (Book)?
Вывести ISBN всех книг (Book), количество страниц (PagesNum) больше, чем в два раза больше среднего количества страниц во всех книгах
Вывести категории, в которых есть подкатегории.
Вывести имена всех авторов (Author), написавших больше всех книг. Считать имена уникальными.
Вывести номера читателей (ReaderNr), которые брали (Borrwing) все книги (Book, не Copy) Марка Твена.
У каких (ISBN) книг (Book) больше, чем одна копия (Copy)?
Вывести 10 самых старых (по PubYear) книг. Если в самом древнем году 10 книг или больше, вывести их все. Если нет, вывести, сколько есть, и дальше выводить все книги из каждого предыдущего года, пока не наберется всего 10 или больше.
Вывести все поддерево подкатегорий категории “Sports”.
Задание 2.
Напишите запросы для следующих действий:

Добавить в таблицу Borrowing запись про то, что ‘John Johnson’ взял книгу c ISBN=123456 и CopyNumber=4.
Удалить все книги с годом публикации больше, чем 2000.
Увеличить дату возврата на 30 дней (просто +30) для всех книг в категории "Databases",  у которых эта дата > '01.01.2022'.

In [ ]:
select title, pubname
from book 

select b.isbn
from book b
where pagesnum = (select max(pagesnum) from book)

select author
from book b 
group by author
having count(b.isbn)>5

select isbn
from book b
where pagesnum > 2*(select avg(pagesnum) from book)

select distinct parentcat 
from category c 
where parentcat is not null 




select author
from book b 
group by author
having count(isbn) = (
	select max(col) from (
	select count(isbn) as col 
	from book 
	group by author 
	)
	)
	
	
select readernr
from borrowing b
join book b2 on b2.isbn=b.isbn
where author = 'Марк Твен'
group by b.readernr
having count(distinct b.isbn) = (
	select count(isbn) 
	from book 
	where author = 'Марк Твен'
	)


select b.isbn
from book b
join copy c on c.isbn = b.isbn
group by b.isbn 
having count(b.isbn)>1


select pubyear   999999
from book b
group by pubyear
order by pubyear 
limit 10


with recursive categotytree as(   10
	select * from category c
	where c.categoryname = 'Sports'
	
	union all
	
	select * 
	from category c
	join categorytree ct on c.parentcat = ct.categoryname
	
)


insert into borrowing (readernr,isbn,copynumber)
values ((select id 
	from reader r 
	where lastname = 'Johnson' and firstname = 'John'),
	123456,4
	)

	
select * from book b

delete from book b 
where pubyear > 2000




update borrowing  set returndate = returndate + interval '30 day'
where returndate > '01.01.2022'

In [ ]:
Определить, во сколько раз зарплата каждого сотрудника меньше максимальной зарплаты по компании.
Определить, во сколько раз зарплата сотрудника отличается от средней зарплаты по департаменту.
Вывести список всех сотрудников. Для каждого сотрудника вывести среднюю зарплату по департаменту и среднюю зарплату по должности. Определить, во сколько раз средняя зарплата по департаменту отличается от средней зарплаты по должности.
Вывести список сотрудников, получающих минимальную зарплату по департаменту. Если в каком-либо департаменте несколько сотрудников получают минимальную зарплату, вывести того, чья фамилия идет раньше по алфавиту.
На основе таблицы employees создать таблицу scores c результатами соревнований со следующим маппингом: employee_id -> man_id, department_id -> division, salary -> score. Вывести список людей, занявших первые 3 места в каждом дивизионе (т.е. занявших три позиции с максимальным количеством очков).
[+1 балл]

6. Отсортировать список сотрудников по фамилиям и разбить на 5 по возможности равных групп. Для каждого сотрудника вывести разницу между его зарплатой и средней зарплатой по группе.

[+1 балл]

7. Для каждого сотрудника посчитать количество сотрудников, принятых на работу в период ± 1 год от даты его принятия на работу, а также количество сотрудников, принятых позже данного сотрудника, но в этом же году. Если два сотрудника приняты в один день, считать принятым позже сотрудника с большим employee_id.